# RTAB-Map ROS MicroCourse

<img src="img/title.png" width="700" />

## Chapter 2: Autonomous Navigation with RTAB-Map

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: <b>40 min</b><br><br>
This Chapter will show how to create a 3D representation of an environment, and how to perform Autonomous Navigation using the rtabmap_ros package.

<p style="background:green;color:white;">END OF SUMMARY</p>

Simultaneous Localization and Mapping (<b>SLAM</b>) is the name that defines the robotic problem of <b>building a map of an unknown environment while simoultaneously keeping track of the robot's location in the map that is being built</b>. As you may already know, in ROS Navigation you use the slam_gmapping package in order to solve this problem. The slam_gmapping package is based on the OpenSLAM implementation, which uses a laser scanner in order to build a 2D map of the environment, and a particle filter in order to be able to localize the robot in the environment. Great, but... was does this have to do with the current Course?
<br><br>
Well, not much actually, because within this Unit, we will be using a completely different SLAM approach. But I just wanted to refresh a little bit some of the knowledge you may already have about SLAM.
<br><br>
The rtabmap_ros package uses an <b>RGB-D SLAM</b> approach based on a global <b>loop closure detector</b>. And... what does this mean? That's a great question! But... before explaining how this actually works, let's first go for a Mapping session.

## Mapping Mode

So, now that you have already seen how to properly launch the rabmap_ros package, it is time to give it some use!! And the first thing we are going to do is to generate a Map of the environment. So... let's go! For that, you'll need to create a new launch file in order to launch both RTAB-Map and the Navigation system.
<br><br>
For building this new launch file, you should take these 3 things into account:
<ol>
<li>Regarding the Navigation System, you will just need to launch the <i>move_base</i> node, since the SLAM process will be handled by the <i>rtabmap_ros</i> package itself.</li>
<br>
<li>By default, the rtabmap_ros package publishes the grid map that it's created into a <i>/grid_map</i> topic. In the Navigation system, though, the grid map is read from the <i>/map topic</i>. So, you'll need to do the propper remap here.</li>
</ol>

<p style="background:#EE9023;color:white;">Exercise 1.1</p>
<br>

a) In your previously created package, modify the launch file so that it properly launches the rtabmap_ros package alongside with the navigation system.
<br><br>
b) Launch this file and check that everything went OK.
<br><br>
c) Execute the following command in order to be able to visualize the mapping process with RViz.


d) Move the robot around the environment in order to generate a Map. To teleoperate the robot, use the following command:

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.1</p>
<br>
Check the following Notes in order to complete the Example:
<br><br>
<span style="color:orange">Note 1: </span>To launch the move_base node, you can include the move_base_rtab.launch.xml file from the turtlebot_navigation package.</i><br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Exercise 1.1</p>

<img src="img/3d_mapping.png" width="800" />

<img src="img/3d_mapping_up.png" width="800" />

<p style="color:red;"><b>NOTE: Please, do not look to this until you get really stucked.</b></p>

Complete Launch File:

In [ ]:
<launch>
  
  <arg name="database_path"     default="rtabmap.db"/>
  <arg name="args"              default=""/>
  
  <arg name="wait_for_transform"  default="0.2"/> 
  
  <!-- Navigation stuff (move_base) -->
  <include file="$(find turtlebot_navigation)/launch/includes/move_base_rtab.launch.xml"/>
  
  <!-- Mapping -->
  <group ns="rtabmap">

    <node name="rtabmap" pkg="rtabmap_ros" type="rtabmap" output="screen" args="$(arg args)">
      <param name="database_path"       type="string" value="$(arg database_path)"/>
      <param name="frame_id"            type="string" value="base_footprint"/>
      <param name="odom_frame_id"       type="string" value="odom"/>
      <param name="wait_for_transform_duration"  type="double"   value="$(arg wait_for_transform)"/>
      <param name="subscribe_depth"     type="bool"   value="true"/>
      <param name="subscribe_scan"      type="bool"   value="true"/>
    
      <!-- inputs -->
      <remap from="scan"            to="/kobuki/laser/scan"/>
      <remap from="rgb/image"       to="/camera/rgb/image_raw"/>
      <remap from="depth/image"     to="/camera/depth/image_raw"/>
      <remap from="rgb/camera_info" to="/camera/rgb/camera_info"/>
      
      <!-- output -->
      <remap from="grid_map" to="/map"/>
    
      <!-- RTAB-Map's parameters: do "rosrun rtabmap rtabmap (double-dash)params" to see the list of available parameters. -->
      <param name="RGBD/ProximityBySpace"        type="string" value="true"/>   
      <param name="RGBD/OptimizeFromGraphEnd"    type="string" value="false"/>  
      <param name="Kp/MaxDepth"                  type="string" value="4.0"/>
      <param name="Reg/Strategy"                 type="string" value="1"/>      
      <param name="Icp/CoprrespondenceRatio"     type="string" value="0.3"/>
      <param name="Vis/MinInliers"               type="string" value="5"/>      
      <param name="Vis/InlierDistance"           type="string" value="0.1"/>    
      <param name="RGBD/AngularUpdate"           type="string" value="0.1"/>    
      <param name="RGBD/LinearUpdate"            type="string" value="0.1"/>    
      <param name="Rtabmap/TimeThr"              type="string" value="700"/>
      <param name="Mem/RehearsalSimilarity"      type="string" value="0.30"/>
      <param name="Optimizer/Slam2D"             type="string" value="true"/>
      <param name="Reg/Force3DoF"                type="string" value="true"/>   
      
    </node>
   
  </group>
</launch>

<p style="background:#EE9023;color:white;">End of Example 1.1</p>

After a mapping session as above, a database is saved here ~/.ros/rtabmap.db. Into this database, the rtabmap_ros package stores, for instance, images from the mapping session that will be later used for <b>detecting loop closures</b>.

In order to better explain how this works, let's access to this database. The rtabmap_ros package offers a tool that allows us to visualize the content of this database: the <b>RTAB-Map's Databae Viewer</b>. You can open this tool using the following command:

You will then see something like this:
<br>
<img src="img/database1.png" width="500" />

<p style="color:red;"><b>NOTE: Remember to open the Graphic Interface in order to visualize the tool!</b></p>

This images you are seeing right now are the different images that have been taken during the Mapping session. If you move the "Id" scroll button that appears at the bottom of the screen, you will move through all the different images that are stored in the database.

But, as you may have noticed, most of the images have some strange yellow marks on them (for now, forget about the pink marks). What are these strange yellow marks? Can you guess? Well, basically, <b>this overlapping yellow disks are marking/highlightning the key features of each image</b>. And... how are the key features of each image selected? That's another great question!
<br><br>
This visual features used by RTAB-Map are using some popular techniques from computer vision including like SIFT, SURF, BRIEF, FAST, BRISK, ORB or FREAK.  Most of these algorithms look for large changes in intensity in different directions around a point in the image.  If you check into the different images, you will notice that there are no yellow discs centered on the homogeneous parts of the image such as the walls or the floor.  Instead, the discs are inserted into areas where there are changes in intensity such as the corners. Corner-like features tend to be stable properties of a given location and can be easily detected even under different lighting conditions or when the robot’s view is from a different angle or distance from an object.

The rtabmap_ros package records these collections of visual features in memory as the robot maps the area.  At the same time, a machine learning technique known as the “bag of words model” looks for patterns in the features that can then be used to classify the various images as belonging to one location.  For instance, there may be a hundred different video frames like the one shown above but from slightly different viewpoints that all contain visual features similar enough to assign to the same location.

That's awesome, right? But now... what are those pink discs that appear in some cases? Could you find out what they mean? Can you detect in which cases they appear? I'll give you some minutes...

So what? Did you discovered anything? Well... let's solve the mystery! The <b>pink discs indicate visual features that two images have in common</b>. For instance, if you select in the Database Viewer 2 images with the same Id (so that are the same image), there should be lots of visual features in common between both images, right?
<br>
<img src="img/same_id.png" width="400" />
<br>
On the other hand, if we select 2 images that don't have key features in common, we won't get any pink disk.
<br>
<img src="img/diff_id.png" width="400" />
<br>
Based on the number of shared features and their geometric relations to one another, we can determine if the two views should be assigned to the same location or not.  In this way, only a subset of the visual features needs to be stored in long term memory while still being able to recognize a location from many different viewpoints.  As a result, RTAB-Map can map out large areas such as an entire building or an outdoor campus without requiring an excessive amount of memory storage or processing power to create or use the map.
<br><br>
So... what do you say? Amazing, right? Well, now you know a little bit better how the whole process works, let's move to the Localization section!

## Localization Mode

So, after we have already mapped the environment, we can then relaunch the rtabmap_ros package within the localization mode.

In order to launch the package in localization mode, you need to take into account the following:
<uol>
<li>The RTAB-Map Parameter <i>Mem/IncrementalMemory</i> has to be set to false, and <i>Mem/InitWMWithAllNodes</i> has to be set to true.</li>
</ul>

<p style="color:green;"><b>NOTE: The recommended way of doing this is adding to the previous launch file the required aprameters for the localization mode, and then adding a condition to them. See the following example:</b></p>

This way, you can launch the localization mode by just adding a parameter to the launch file you've already created. Like this:

When the localization mode is launched, you can move the robot around the environment until it can relocalize in the previous map. Then, <b>the 2D map would re-appear again when a loop closure is found</b>.

<p style="background:#EE9023;color:white;">Exercise 1.2</p>
<br>

a) Make the necessary modifications to the launch file you've created in Exercise 1.1.
<br><br>
b) Stop the RViz session you launched in Exercise 1.1, and move the robot to a random location on the map.
<br><br>
c) Launch the rtabmap package in Localization mode.

d) Launch again RViz and move the robot around the environment until it relocalizes again.

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.2</p>
<br>
Check the following Notes in order to complete the Example:
<br><br>
<span style="color:orange">Note 1: </span>In order to be able to relocalie itself, the package must detect a loop closure. So, you should move the robot to a place where you know your database has recognizable images from.</i><br>
<span style="color:orange">Note 2: </span>When your robot localizes itself and the whole 2D map appears, if the 3D map doesn't appear, you can just click on the "Download map" option in Rtabmap cloud panel.</i><br>
<img src="img/rtabmap_cloud.png" width="200" />
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Exercise 1.2</p>

Robot without localizing himself:

<img src="img/localization1.png" width="600" />

Robot localized:

<img src="img/localization2.png" width="600" />

<p style="color:red;"><b>NOTE: Please, do not look to this until you get really stucked.</b></p>

Complete Launch File:

In [ ]:
<launch>
  
  <arg name="database_path"     default="rtabmap.db"/>
  <arg name="localization"      default="false"/>
  <arg name="args"              default=""/>
  
  <arg name="wait_for_transform"  default="0.2"/> 
  
  <!-- Navigation stuff (move_base) -->
  <include file="$(find turtlebot_navigation)/launch/includes/move_base_rtab.launch.xml"/>
  
  <!-- Mapping -->
  <group ns="rtabmap">

    <node name="rtabmap" pkg="rtabmap_ros" type="rtabmap" output="screen" args="$(arg args)">
      <param name="database_path"       type="string" value="$(arg database_path)"/>
      <param name="frame_id"            type="string" value="base_footprint"/>
      <param name="odom_frame_id"       type="string" value="odom"/>
      <param name="wait_for_transform_duration"  type="double"   value="$(arg wait_for_transform)"/>
      <param name="subscribe_depth"     type="bool"   value="true"/>
      <param name="subscribe_scan"      type="bool"   value="true"/>
    
      <!-- inputs -->
      <remap from="scan"            to="/kobuki/laser/scan"/>
      <remap from="rgb/image"       to="/camera/rgb/image_raw"/>
      <remap from="depth/image"     to="/camera/depth/image_raw"/>
      <remap from="rgb/camera_info" to="/camera/rgb/camera_info"/>
      
      <!-- output -->
      <remap from="grid_map" to="/map"/>
    
      <!-- RTAB-Map's parameters: do "rosrun rtabmap rtabmap (double-dash)params" to see the list of available parameters. -->
      <param name="RGBD/ProximityBySpace"        type="string" value="true"/>   
      <param name="RGBD/OptimizeFromGraphEnd"    type="string" value="false"/>  
      <param name="Kp/MaxDepth"                  type="string" value="4.0"/>
      <param name="Reg/Strategy"                 type="string" value="1"/>      
      <param name="Icp/CoprrespondenceRatio"     type="string" value="0.3"/>
      <param name="Vis/MinInliers"               type="string" value="5"/>      
      <param name="Vis/InlierDistance"           type="string" value="0.1"/>    
      <param name="RGBD/AngularUpdate"           type="string" value="0.1"/>    
      <param name="RGBD/LinearUpdate"            type="string" value="0.1"/>    
      <param name="Rtabmap/TimeThr"              type="string" value="700"/>
      <param name="Mem/RehearsalSimilarity"      type="string" value="0.30"/>
      <param name="Optimizer/Slam2D"             type="string" value="true"/>
      <param name="Reg/Force3DoF"                type="string" value="true"/>   
      
      <!-- localization mode -->
      <param     if="$(arg localization)" name="Mem/IncrementalMemory" type="string" value="false"/>
      <param unless="$(arg localization)" name="Mem/IncrementalMemory" type="string" value="true"/>
      <param name="Mem/InitWMWithAllNodes" type="string" value="$(arg localization)"/> 
    </node>
   
  </group>
</launch>


<p style="background:#EE9023;color:white;">End of Exercise 1.2</p>

## Autonomous Navigation

So now... you're ready to autonomously navigate your robot around the map!
<br><br>
In order to make the robot move, you just need to send goals to the <i>/move_base/goal</i> topic, as you would normally do when using the Navigation stack. If you want to send goals through Rviz, you just have to use the <i>2D Nav Goal</i> option as shown in the picture below: 

<img src="img/sending_goal.png" width="600" />

<p style="background:#EE9023;color:white;">Exercise 1.3</p>
<br>
a) Move the robot to a new random location in the map.
<br><br>
b) Launch the rtabmap package in Localization mode.
<br><br>
c) Move the robot around unitl it localizes itself, just as you did in the previous exercise.
<br><br>
d) Now, you're ready to start sending goals to the move_base node!

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.3</p>
<br>
Check the following Notes in order to complete the Example:
<br><br>
<span style="color:orange">Note 1: </span>Autonomous Navigation won't work until the robot localizes itself correctly. This means, until it finds a loop closure.</i><br>
<span style="color:orange">Note 2: </span>Make sure that you are not teleoperating the robot. If you have the keyboard_teleop.launch launched, autonomous navigation may not work.<br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Exercise 1.3</p>

<img src="img/goal_reached.png" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 1.3</p>